<a href="https://colab.research.google.com/github/Pansyhb/Git-6300/blob/main/Copy_of_Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Creation and Training

In [ ]:
!pip install yfinance
!pip install findspark
!pip install pyspark
!pip install vaderSentiment
import pandas as pd
import csv
import datetime
import yfinance as yf
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup as bs
import requests
from pyspark.sql.functions import sum,max,min,mean,count
import datetime as dt
import pyspark
from pyspark.sql import SparkSession
import findspark
import yaml
from yaml.loader import SafeLoader
from os.path import abspath
!git clone https://github.com/Pansyhb/Git-6300.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=725ac28518228ab7f0a285428d67230f8ace4ede2284b8a1f6b63ac75b332636
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://

##Read in CSV Files Containing Data for Target Companies

In [ ]:
from os import listdir
import pandas as pd
path = '/content/Git-6300/data/data/'

from sklearn.preprocessing import MinMaxScaler
#Function to read in csv files and scale columns for data where scaling of predictions is eventually needed
def scale(path):
    scaler_list = []
    target_list = []
    feat_list = []
    for item in listdir(path):
        print(str(item))
        df = pd.read_csv(path + str(item))
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        print(df)
        #create scaler instances
        scaler = MinMaxScaler(feature_range=(0,1))
        target_scaler = MinMaxScaler(feature_range = (0,1))
        data = df.drop(['ticker'],axis=1)
        # scale features and target columns
        target = target_scaler.fit_transform(data[['target']])
        target = target.flatten()
        scaler_list.append(target_scaler)
        target_list.append(target)
        X_feat = data.drop(['target'], axis = 1)
        for col in X_feat.columns:
            X_feat[col] = scaler.fit_transform(X_feat[[col]])
        feat_list.append(X_feat)
    return scaler_list, target_list, feat_list
scaler_list, target_list, feat_list = scale(path) 

MSFT_dataframe.csv
           ticker        open        high         low       close   adj_close  \
date                                                                            
2016-01-04   MSFT   54.320000   54.799999   53.389999   54.799999   49.007950   
2016-01-05   MSFT   54.930000   55.389999   54.540001   55.049999   49.231518   
2016-01-06   MSFT   54.320000   54.400002   53.639999   54.049999   48.337219   
2016-01-07   MSFT   52.700001   53.490002   52.070000   52.169998   46.655918   
2016-01-08   MSFT   52.369999   53.279999   52.150002   52.330002   46.799000   
...           ...         ...         ...         ...         ...         ...   
2023-02-23   MSFT  255.559998  256.839996  250.479996  254.770004  254.770004   
2023-02-24   MSFT  249.960007  251.000000  248.100006  249.220001  249.220001   
2023-02-27   MSFT  252.460007  252.820007  249.389999  250.160004  250.160004   
2023-02-28   MSFT  249.070007  251.490005  248.729996  249.419998  249.419998   
2023-03-0

In [ ]:
#Split training and test data (microsoft and google) was well as features and targets
test_data = feat_list[0]   
test_target = target_list[0]
train_data = feat_list[1] 
train_target = target_list[1]
print(train_data.reset_index().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1802 non-null   datetime64[ns]
 1   open        1802 non-null   float64       
 2   high        1802 non-null   float64       
 3   low         1802 non-null   float64       
 4   close       1802 non-null   float64       
 5   adj_close   1802 non-null   float64       
 6   volume      1802 non-null   float64       
 7   tenmda      1802 non-null   float64       
 8   twentymda   1802 non-null   float64       
 9   fiftymda    1802 non-null   float64       
 10  hundredmda  1802 non-null   float64       
 11  EWMA_20     1802 non-null   float64       
 12  EWMA_50     1802 non-null   float64       
 13  EWMA_100    1802 non-null   float64       
 14  rsi         1802 non-null   float64       
 15  MiddleBand  1802 non-null   float64       
 16  UpperBand   1802 non-nul

In [ ]:
# Split data into X_train and y_train data sets
import numpy as np
def lstm_split(data,target,steps):
      X = []
      y = []
      # Creating a data structure with 10 time-steps and 1 output
      for i in range(10, steps):
          X.append(data[i-10:i])
          y.append(target[i])  
      return np.array(X),np.array(y)
 
X1,y1 = lstm_split(train_data, train_target,len(train_data))

#Define Training and Test Datasets
def train_split(x1, y1, train_data):
  train_split = 0.9
  split_idx = int(np.ceil(len(X1)*train_split))
  date_index = train_data.index
 
  X_train,X_test = X1[:split_idx],X1[split_idx:]
  y_train,y_test = y1[:split_idx],y1[split_idx:]
  X_train_date,X_test_date = date_index[:split_idx],date_index[split_idx:]
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_split(X1, y1, train_data)
print(X1.shape,X_train.shape,X_test.shape,y_test.shape,y_train.shape)

(1792, 10, 18) (1613, 10, 18) (179, 10, 18) (179,) (1613,)


#Configure the base LSTM model 
This process involved running a random search function to select best hyperparameters for the network. The results of this search are availble below.


In [ ]:
!pip install tensorflow
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.metrics import Precision
from keras.optimizers import Adam
!pip install keras_tuner
import keras_tuner

#define function to create model, optional hyperparameters included to be selected during training
LR = 0.05
def build_model(hp):
  model = Sequential()
  hidden = hp.Choice('n_hidden', [0,1,2,3])
  model.add(LSTM(units = hp.Int('neurons_visible', min_value = X_train.shape[2], max_value = 100, step = 20),
                activation = hp.Choice('activate1', ['sigmoid', 'relu']),
                input_shape = (X_train.shape[1], X_train.shape[2]),
                return_sequences = True if hidden >0 else False))
  #Configure hidden layers based on random search determined hidden layer number
  if hidden > 0:
    for num in range(hidden):
      model.add(Dropout(hp.Float('dropout' +str(num+1), min_value = 0.1, max_value = 0.9, step = 0.3)))
      model.add(LSTM(units = hp.Int('neurons_hidden'+str(num+1), min_value = 20, max_value = 50, step = 10),
                     activation = 'relu', return_sequences = True if num != hidden else False))
      
  model.add(Dropout(0.5))
  model.add(Dense(units = 1, activation = 'sigmoid'))

  #compile the model
  model.compile(loss = 'mean_squared_error', optimizer = Adam(lr = LR), metrics = ['mean_squared_error'])
  
  return model

#set learning rate and early stopping callbacks
LR_decay = ReduceLROnPlateau('loss', patience=1, verbose=0, 
                             factor=0.5, min_lr=1e-8)
Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                           patience=25, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)
#arrange random search class
tune = keras_tuner.RandomSearch(build_model, objective = 'val_loss', max_trials = 50, seed = 1)

#complete training 
tune.search(X_train, y_train, epochs = 200, batch_size = 20, validation_data = (X_test, y_test), callbacks = [LR_decay, Early_stop])

LSTM_model = tune.get_best_models()[0]
LSTM_model.save('main_models/LSTM_model1.h5')
hyperparameters = tune.get_best_hyperparameters()[0]
print(hyperparameters.values)

Trial 50 Complete [00h 00m 38s]
val_loss: 0.07142400741577148

Best val_loss So Far: 0.0011959553230553865
Total elapsed time: 00h 37m 53s
{'n_hidden': 0, 'neurons_visible': 78, 'activate1': 'sigmoid', 'dropout1': 0.7, 'neurons_hidden1': 50, 'dropout2': 0.4, 'neurons_hidden2': 30, 'dropout3': 0.1, 'neurons_hidden3': 40}


## Configure the Prophet Model

In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import ParameterGrid
!pip install yfinance
import yfinance as yf
import datetime as dt

#Generate basic financial data from yfinance for prophet training
def prophet_data(ticker, start_day, month, year):
    delta = dt.timedelta(days = 150)
    data = yf.download(ticker, (dt.date(year, month, start_day) - delta)).reset_index()
    print(data.head())
    data = data.rename(columns = {'Date':'ds', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Adj Close': 'y', 'Volume':'volume'})
    print(data.head())
    data = data.loc[:,['ds', 'y']]
    return data

MSFT_data = prophet_data('MSFT',1, 1,2016)
GOOG_data = prophet_data('GOOG', 1, 1, 2016)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed
        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-08-04  46.750000  47.709999  46.680000  47.540001  41.953976  33403900
1 2015-08-05  47.980000  48.410000  47.540001  47.580002  41.989269  26959700
2 2015-08-06  47.709999  47.770000  46.330002  46.619999  41.142078  27368000
3 2015-08-07  46.389999  46.779999  46.259998  46.740002  41.247982  19163000
4 2015-08-10  46.950001  47.490002  46.840000  47.330002  41.768665  23079900
          ds       open       high        low      close          y    volume
0 2015-08-04  46.750000  47.709999  46.680000  47.540001  41.953976  33403900
1 2015-08-05  47.980000  48.410000  47.540001  47.580002  41.989269  26959700
2 2015-08-06  47.709999  47.770000  46.330002  46.619999  41.142078  27368000
3 2015-08-07  46.389999  46.779999  46.259998  46.74

In [ ]:


# Define parameter grid to search over
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.01, 0.1, 1.0],
    'seasonality_prior_scale': [0.01, 0.1, 1.0],
}

# Initialize minimum error and best parameters
min_error = float('inf')
best_params = {}

# Loop through all parameter combinations
for params in ParameterGrid(param_grid):
    print('Testing parameters:', params)

    # Initialize Prophet model with specified hyperparameters
    model = Prophet(**params)
    model.fit(MSFT_data)

    # Perform time series cross-validation
    df_cv = cross_validation(model=model, initial='1000 days', horizon='10 days', period='10 days')

    # Calculate performance metrics
    df_metrics = performance_metrics(df_cv)

    # Calculate mean cross-validation error
    mean_cv_error = df_metrics['mse'].mean()

    # Update minimum error and best parameters if new minimum is found
    if mean_cv_error < min_error:
        min_error = mean_cv_error
        best_params = params

# Print best hyperparameters and corresponding error
print('Best parameters:', best_params)
print('Minimum cross-validation error:', min_error)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed
        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-09-23  43.930000  44.169998  43.509998  43.869999  38.970501  17145200
1 2015-09-24  43.450001  44.130001  43.270000  43.910000  39.006042  27905600
2 2015-09-25  44.480000  44.730000  43.759998  43.939999  39.032688  29384600
3 2015-09-28  43.830002  44.090000  43.209999  43.290001  38.455292  27613800
4 2015-09-29  43.369999  43.570000  43.049999  43.439999  38.588528  32763600
          ds       open       high        low      close          y    volume
0 2015-09-23  43.930000  44.169998  43.509998  43.869999  38.970501  17145200
1 2015-09-24  43.450001  44.130001  43.270000  43.910000  39.006042  27905600
2 2015-09-25  44.480000  44.730000  43.759998  43.939999  39.032688  29384600
3 2015-09-28  43.830002  44.090000  43.209999  43.29

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/l43gz6to.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/5z02mzsv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=80081', 'data', 'file=/tmp/tmpn3ekw2r5/l43gz6to.json', 'init=/tmp/tmpn3ekw2r5/5z02mzsv.json', 'output', 'file=/tmp/tmpn3ekw2r5/prophet_model8b4m1ju7/prophet_model-20230329163759.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:37:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:38:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 174 forecasts with cutoffs between 2018-06-23 00:00:00 and 2023-03-19 00:00:00


  0%|          | 0/174 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/kq4iku4x.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/xe1tq_76.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19841', 'data', 'file=/tmp/tmpn3ekw2r5/kq4iku4x.json', 'init=/tmp/tmpn3ekw2r5/xe1tq_76.json', 'output', 'file=/tmp/tmpn3ekw2r5/prophet_modelmn0iazq0/prophet_model-20230329163800.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:38:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:38:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/1oqihqik.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/h7lc02h5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

Testing parameters: {'changepoint_prior_scale': 0.01, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 0.1}


...

Testing parameters: {'changepoint_prior_scale': 1.0, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 1.0}


DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75018', 'data', 'file=/tmp/tmpn3ekw2r5/wvqlcrdp.json', 'init=/tmp/tmpn3ekw2r5/kwh2vpy2.json', 'output', 'file=/tmp/tmpn3ekw2r5/prophet_modeli3qzmigd/prophet_model-20230329174211.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:42:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:42:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 174 forecasts with cutoffs between 2018-06-23 00:00:00 and 2023-03-19 00:00:00


  0%|          | 0/174 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/2t4ig3ni.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/mwdkc_du.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97011', 'data', 'file=/tmp/tmpn3ekw2r5/2t4ig3ni.json', 'init=/tmp/tmpn3ekw2r5/mwdkc_du.json', 'output', 'file=/tmp/tmpn3ekw2r5/prophet_modeleb5vwq4v/prophet_model-20230329174214.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:42:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:42:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/8rqee6i0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpn3ekw2r5/k41v7ae9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

Best parameters: {'changepoint_prior_scale': 1.0, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 0.01}
Minimum cross-validation error: 381.8763341337825


## Generate predictions on test data

In [ ]:
# We use the best parameter to fit the model
def get_data(df):
  Best_parameters={'changepoint_prior_scale': 0.1, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 0.01}
  # We use the best parameter to fit the model
  final_model = Prophet(**Best_parameters)
  final_model.fit(df)
  df = cross_validation(model=final_model, initial='100 days', horizon='10 days', period='10 days')
  return df

#generate msft and google prediction datasets 
GOOG_data = get_data(GOOG_data)[['ds', 'yhat']]
MSFT_data = get_data(MSFT_data)[['ds', 'yhat']]


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/_d5p80wf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/2is830dk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20502', 'data', 'file=/tmp/tmpgno32dbc/_d5p80wf.json', 'init=/tmp/tmpgno32dbc/2is830dk.json', 'output', 'file=/tmp/tmpgno32dbc/prophet_modelgwm685wp/prophet_model-20230401161549.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:15:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:15:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 269 forecasts with cutoffs between 2015-11-18 00:00:00 and 2023-03-21 00:00:00


  0%|          | 0/269 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/sqklcinm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/rog6q8dd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56958', 'data', 'file=/tmp/tmpgno32dbc/sqklcinm.json', 'init=/tmp/tmpgno32dbc/rog6q8dd.json', 'output', 'file=/tmp/tmpgno32dbc/prophet_model4ofjlthv/prophet_model-20230401161551.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:15:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:15:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/3cbl42pz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/tw3z_732.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

  0%|          | 0/269 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/4yv5ewet.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/t_qlqz4u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=47289', 'data', 'file=/tmp/tmpgno32dbc/4yv5ewet.json', 'init=/tmp/tmpgno32dbc/t_qlqz4u.json', 'output', 'file=/tmp/tmpgno32dbc/prophet_model7mq6ryio/prophet_model-20230401162037.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:20:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:20:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/7u5gpg1k.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgno32dbc/6ue56qrs.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [ ]:
def prep_prophet(prophet_pred, data1):
  print(prophet_pred.info())
  prophet_pred = prophet_pred.rename(columns = {'ds': 'date'}).set_index('date')
  main_data = data1.join(prophet_pred,how = 'left')
  scale = MinMaxScaler()
  main_data['yhat'] = scale.fit_transform(main_data[['yhat']])
  returned = main_data.loc['2016-01-01':'2023-03-01', 'yhat']
  return returned

Goog_data_proph = prep_prophet(GOOG_data, test_data)[10:]
Msft_train_proph= prep_prophet(MSFT_data, train_data)[10:1623]
Msft_test_proph = prep_prophet(MSFT_data, train_data)[1623:]
print(train_data.loc['2016-01-01':'2023-03-01',].shape)
print(Msft_train_proph)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1853 non-null   datetime64[ns]
 1   yhat    1853 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 29.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1853 non-null   datetime64[ns]
 1   yhat    1853 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 29.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1853 non-null   datetime64[ns]
 1   yhat    1853 non-null   float64       
dtypes: datetime64[ns](1

## Create Dataset to train hybrid model

In [ ]:
from keras.saving.legacy.save import load_model

#create training data using predictions from lstm and prophet
lstm = load_model('main_models/LSTM_model1.h5')
def return_hybrid_data(lstm_model, prophet_data, LSTM_data, LSTM_target):
  preds_dict = {}
  preds_dict['lstm_pred'] = lstm_model.predict(LSTM_data).flatten()
  print(lstm_model.predict(LSTM_data).shape)
  preds_dict['prophet_pred'] = prophet_data
  hybrid_train = pd.DataFrame(preds_dict)
  hybrid_train, target = lstm_split(hybrid_train, LSTM_target, len(hybrid_train))
  return hybrid_train, target

hybrid_train, target = return_hybrid_data(lstm, Msft_train_proph, X_train, y_train)

51/51 [==============================] - 0s 3ms/step
(1613, 1)


## Configure Hybrid Model

In [ ]:
LR = 0.01
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.metrics import Precision
from keras.optimizers import Adam

#General LSTM model based on randomsearch results
def multiple_models(x_train, y_train):
    hybrid = Sequential()
    hybrid.add(LSTM(units = 58, activation = "sigmoid", input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = False))
    hybrid.add(Dropout(0.7))
    hybrid.add(Dense(units =1))
    hybrid.compile(loss = 'mean_squared_error', optimizer = Adam(lr = LR), metrics = ["mean_squared_error"])
    Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                              patience=25, verbose=1, mode='auto',
                              baseline=0, restore_best_weights=True)
    hybrid.fit(x_train, y_train, epochs = 200, batch_size = 20, callbacks = [Early_stop])
    return hybrid
    
#train the hybrid model
hybrid1 = multiple_models(hybrid_train, y_train[10:])
hybrid1.save('main_models/hybrid1.h5')

Epoch 1/200


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


81/81 [==============================] - 2s 8ms/step - loss: 0.2397 - mean_squared_error: 0.2397
Epoch 2/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0642 - mean_squared_error: 0.0642
Epoch 3/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0313 - mean_squared_error: 0.0313
Epoch 4/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0197 - mean_squared_error: 0.0197
Epoch 5/200
81/81 [==============================] - 1s 8ms/step - loss: 0.0188 - mean_squared_error: 0.0188
Epoch 6/200
81/81 [==============================] - 1s 7ms/step - loss: 0.0144 - mean_squared_error: 0.0144
Epoch 7/200
81/81 [==============================] - 1s 7ms/step - loss: 0.0150 - mean_squared_error: 0.0150
Epoch 8/200
81/81 [==============================] - 1s 7ms/step - loss: 0.0128 - mean_squared_error: 0.0128
Epoch 9/200
81/81 [==============================] - 1s 7ms/step - loss: 0.0128 - mean_squared_error: 0.0128
Epoch 10/200
81/81 [========

# Test Single Company Models (LSTM vs. Hybrid)

In [ ]:
#Configure LSTM test using google data and micosoft validation data
goog_test, goog_target = lstm_split(test_data, test_target, len(test_data))
LSTM_predictions = lstm.predict(goog_test)
LSTM_validation = lstm.predict(X_test).flatten()
abs = scaler_list[1].inverse_transform([LSTM_validation]) - scaler_list[1].inverse_transform([y_test])

#Generate google test data for hybrid model

google_ft, google_target = return_hybrid_data(lstm, Goog_data_proph, goog_test, goog_target)
hybrid_val_feat, hybrid_val_target = return_hybrid_data(lstm, Msft_test_proph, X_test, y_test)


6/6 [==============================] - 0s 6ms/step
date
2016-01-19    0.015490
2016-01-20    0.016484
2016-01-21    0.016243
2016-01-22    0.015613
2016-01-25    0.017906
                ...   
2023-02-23    0.477089
2023-02-24    0.474497
2023-02-27    0.467847
2023-02-28    0.465430
2023-03-01    0.464413
Name: yhat, Length: 1792, dtype: float64
56/56 [==============================] - 0s 6ms/step
(1792, 1)
6/6 [==============================] - 0s 4ms/step
(179, 1)
[[ -2.19529961   1.41095637   3.26455313  -1.04150997  -4.08593878
   -3.34428401  -4.07592958  -0.83550844   2.21331312   3.86498477
    6.76880104   5.27610947  -4.19189363  -6.81205875  -7.76970372
   -3.6742091    1.98328643   3.80637403   4.56625121   3.19804319
   -1.1931655    0.7456853   -2.03441845  -1.40989063  -1.79656211
    6.28524615   6.84558613   4.63830286  -5.89369582  -7.03098123
   -7.2369991   -0.25206648   2.16650818  -2.12102996  -3.50785222
   -1.4537286    2.19401272   3.34518034  -1.50743354  -1.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot MAE and loss
#plt.figure(figsize=(10, 6))
plt.plot(hybrid_history.history['mae'], label='MAE')
plt.plot(hybrid_history.history['loss'], label='loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
def compute_metrics(true_series, forecast):
    """Helper to print MSE and MAE"""
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

In [ ]:
mse, mae = compute_metrics(y1, predictions)
print(mae)

In [ ]:
plt.plot(y1, label="actual")
plt.plot(predictions, label="predicted")
plt.xlabel("Timesteps")
plt.ylabel("Value")
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.legend()
plt.show()

# Expanded Model

In [ ]:
LR = 0.01
def multiple_models(x_train, y_train):
    hybrid = Sequential()
    hybrid.add(LSTM(units = 58, activation = "sigmoid", input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = False))
    hybrid.add(Dropout(0.7))
    hybrid.add(Dense(units =1))
    hybrid.compile(loss = 'mean_absolute_error', optimizer = Adam(lr = LR), metrics = ["mean_squared_error"])
    Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                              patience=25, verbose=1, mode='auto',
                              baseline=0, restore_best_weights=True)
    hybrid.fit(x_train, y_train, epochs = 200, batch_size = 20, callbacks = [Early_stop])
    return hybrid
    
def create_data_lists(list_of_feats, list_of_targets):
  training_list = []
  target_list = []
  for data, target in zip(list_of_feats, list_of_targets):
    data_x, data_y = lstm_split(data, target, len(data))
    print(data_x.shape)
    training_list.append(data_x)
    target_list.append(data_y)
  return training_list, target_list

#Train each of the lstm models on training companies
path = '/content/Git-6300/expanded_data/expanded_data/'
ticker_list = ["MSFT", "NFLX", "AMZN", "TSLA"]
hybrid_scalers, targets, frames = scale(path)
training_list, target_list = create_data_lists(frames, targets)

for tick, x_train, y_train in zip(ticker_list, training_list, target_list):
  model = multiple_models(x_train, y_train)
  model.save('sub_models/'+tick+'_lstm.h5')

NFLX_dataframe.csv
           ticker        open        high         low       close   adj_close  \
date                                                                            
2016-01-04   NFLX  109.000000  110.000000  105.209999  109.959999  109.959999   
2016-01-05   NFLX  110.449997  110.580002  105.849998  107.660004  107.660004   
2016-01-06   NFLX  105.290001  117.910004  104.959999  117.680000  117.680000   
2016-01-07   NFLX  116.360001  122.180000  112.290001  114.559998  114.559998   
2016-01-08   NFLX  116.330002  117.720001  111.099998  111.389999  111.389999   
...           ...         ...         ...         ...         ...         ...   
2023-02-23   NFLX  331.230011  331.279999  314.299988  323.649994  323.649994   
2023-02-24   NFLX  319.299988  321.500000  314.519989  317.149994  317.149994   
2023-02-27   NFLX  323.869995  330.000000  322.119995  323.029999  323.029999   
2023-02-28   NFLX  323.700012  327.619995  321.170013  322.130005  322.130005   
2023-03-0

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


90/90 [==============================] - 2s 7ms/step - loss: 0.2320 - mean_squared_error: 0.1225
Epoch 2/200
90/90 [==============================] - 1s 9ms/step - loss: 0.1039 - mean_squared_error: 0.0182
Epoch 3/200
90/90 [==============================] - 1s 13ms/step - loss: 0.0823 - mean_squared_error: 0.0115
Epoch 4/200
90/90 [==============================] - 1s 11ms/step - loss: 0.0799 - mean_squared_error: 0.0109
Epoch 5/200
90/90 [==============================] - 1s 9ms/step - loss: 0.0743 - mean_squared_error: 0.0099
Epoch 6/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0737 - mean_squared_error: 0.0095
Epoch 7/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0702 - mean_squared_error: 0.0090
Epoch 8/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0686 - mean_squared_error: 0.0082
Epoch 9/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0689 - mean_squared_error: 0.0084
Epoch 10/200
90/90 [=========

## Train Expanded Hybrid Model Using Predictions of all Sub-Models

In [ ]:
#Create Train Data for Hybrid Model Using Microsoft Predictions
from keras.saving.legacy.save import load_model
import os
def hybrid_trainer(direct, ticker_list, hybrid_train):
    preds_dict = {}
    path = direct
    for num, model in enumerate(os.listdir(path)):
        model = load_model(path+model)
        prediction = model.predict(hybrid_train).flatten()
        print(prediction.shape)
        preds_dict[ticker_list[num]] = prediction
    preds_df = pd.DataFrame(preds_dict)
    return preds_df

#generate training data for expanded hybrid model
hybrid_training_data =  hybrid_trainer('sub_models/', ticker_list, X_train) 
hybrid_training_data['prophet'] = np.array(Msft_train_proph)
hybrid_training_data, y_train = lstm_split(hybrid_training_data, target_list[0], len(hybrid_training_data))


In [ ]:
#Create and train the expanded hybrid model
hybrid_expanded = multiple_models(hybrid_training_data, y_train)
hybrid_expanded.save('main_models/hybrid_expanded.h5')

## Test the Expanded Hybrid Model on Microsoft and Goole Test Data


In [ ]:
#Configure hybrid model testing data based on predictions from each of the four underlying models and prophet
from os import listdir
google_test = {}
microsoft_test = {}
path = 'sub_models/'
for num, model in enumerate(os.listdir(path)):
  model = load_model(path+model)
  print(goog_target)
  goog_preds = model.predict(goog_test).flatten()
  micro_preds = model.predict(X_test).flatten()
  google_test[ticker_list[num]] = goog_preds
  microsoft_test[ticker_list[num]] = micro_preds
google_test = pd.DataFrame(google_test)
microsoft_test = pd.DataFrame(microsoft_test)

#set Prophet data
google_test['prophet'] = Goog_data_proph
microsoft_test['prophet'] = Msft_test_proph

#Create final test data for hybrid model
google_test = lstm_split(goog_test, goog_target, len(google_test))
microsoft_test = lstm_split(microsoft_test, y_test, len(microsoft_test))


[0.00800869 0.0069567  0.01309896 ... 0.68206193 0.67137238 0.68779697]
6/6 [==============================] - 0s 3ms/step
[0.00800869 0.0069567  0.01309896 ... 0.68206193 0.67137238 0.68779697]
6/6 [==============================] - 0s 3ms/step
[0.00800869 0.0069567  0.01309896 ... 0.68206193 0.67137238 0.68779697]
6/6 [==============================] - 0s 4ms/step
[0.00800869 0.0069567  0.01309896 ... 0.68206193 0.67137238 0.68779697]
6/6 [==============================] - 0s 4ms/step


In [ ]:
#Predict google and microsoft test data using the trained hybrid model
hybrid_expanded = load_model('main_models/hybrid_expanded.h5')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot MAE and loss
#plt.figure(figsize=(10, 6))
plt.plot(hybrid_history.history['mae'], label='MAE')
plt.plot(hybrid_history.history['loss'], label='loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

def compute_metrics(true_series, forecast):
    """Helper to print MSE and MAE"""
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

mse, mae = compute_metrics(y1, predictions)
print(mae)
plt.plot(y1, label="actual")
plt.plot(predictions, label="predicted")
plt.xlabel("Timesteps")
plt.ylabel("Value")
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.legend()
plt.show()